In [1]:
import os
import cv2
import json
import glob
import numpy as np
from tqdm import tqdm

In [2]:
videos_dir = '/kaggle/input/aic-videob3-2'
all_video_paths = dict()
for part in sorted(os.listdir(videos_dir)):
    data_part = part.split('_')[-1] # L01, L02 for ex
    all_video_paths[data_part] =  dict()
    
# del all_video_paths['L32']
del all_video_paths['L33']
del all_video_paths['L34']
del all_video_paths['L35']
del all_video_paths['L36']

    
print(all_video_paths)
for data_part in sorted(all_video_paths.keys()):
    data_part_path = f'{videos_dir}/Videos_{data_part}/video'
    video_paths = sorted(os.listdir(data_part_path))
    video_ids = [video_path.replace('.mp4', '').split('_')[-1] for video_path in video_paths]
    for video_id, video_path in zip(video_ids, video_paths):
        video_path_full = f'{data_part_path}/{video_path}'
        all_video_paths[data_part][video_id] = video_path_full

{'L32': {}}


In [3]:
def sample_frames_from_shot(start_idx, end_idx):
    '''
    intervals = np.linspace(start=start_idx, stop=end_idx, num=n_frames+1).astype(int)
    ranges = []
    for idx, interv in enumerate(intervals[:-1]):
        ranges.append((interv, intervals[idx + 1]))
    frame_idxs = [(x[0] + x[1]) // 2 for x in ranges]
    '''
    idx_first = start_idx
    idx_end = end_idx
    idx_mid = start_idx + int((end_idx-start_idx)/2)
    frame_idxs = [idx_first, idx_mid, idx_end]
    return frame_idxs

In [4]:
scene_json_dirs = '/kaggle/input/scenejson/SceneJSON'
save_dir_all = './Keyframes'
if not os.path.exists(save_dir_all):
    os.mkdir(save_dir_all)

for key in all_video_paths.keys():
    save_dir = f'{save_dir_all}/{key}_extra'

    if not os.path.exists(save_dir):
        os.mkdir(save_dir)
    
    video_paths_dict = all_video_paths[key]
    video_ids = sorted(video_paths_dict.keys())
    for video_id in tqdm(video_ids):
        video_path = video_paths_dict[video_id]
        video_scene_path =  f'{scene_json_dirs}/{key}/{video_id}.json'
        
        with open(video_scene_path, 'r') as f:
            video_scenes = json.load(f)
        
        if not os.path.exists(f'{save_dir}/{video_id}'):
            os.mkdir(f'{save_dir}/{video_id}')
        
        cap = cv2.VideoCapture(video_path)
        for i, shot in enumerate(tqdm(video_scenes)):
            shot_frames_id = sample_frames_from_shot(shot[0], shot[1])
            for index in shot_frames_id:
                cap.set(cv2.CAP_PROP_POS_FRAMES, index)
                filename = "{}/{:0>6d}.jpg".format(f'{save_dir}/{video_id}', index)
                ret, frame = cap.read()
                if ret:
                    if not cv2.imwrite(filename, frame):
                        print('fail save')
                else:
                    pass
        cap.release()

100%|██████████| 30/30 [1:23:04<00:00, 166.16s/it]
